<a href="https://colab.research.google.com/github/ashikita/pdf_adjustment/blob/main/pdf_frame_erase_slide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# -*- coding: utf-8 -*-
"""
PDFに対して、枠消し(白塗り)を適用した後、本文を左右方向にスライドして出力します。
・入力ファイル名は設定ブロックで指定（既定: sample.pdf）
・出力ファイル名は [入力ファイル名]_rev.pdf
・枠消し幅は奇数/偶数ページの上下左右（計8変数）を mm 単位で設定可能
・横方向のスライド量は奇数/偶数ページごとに mm 単位で設定可能（+右 / -左）
・傾き調整(デスキュー)は行いません
・RAM節約と速度向上のため、ページごとに逐次処理し、ラスタライズは行いません（テキストの鮮明さを維持）

Jupyterローカルでの実行を想定。必要に応じてPyMuPDFをpipインストールします。
"""

import sys
import time
import os
import gc

# ---- 依存パッケージ（必要ならpip）----
try:
    import fitz  # PyMuPDF
except ImportError:
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--upgrade", "pymupdf"])
    import fitz


# ============================================================
# ★★ 設定ブロック（ここだけ編集すれば使えます） ★★
# ============================================================

# 入力ファイル名（既定: sample.pdf）
INPUT_FILENAME = "sample.pdf"

# 枠消し幅（mm）：奇数ページ（1,3,5,...）
ODD_TOP_MM    = 0.0
ODD_BOTTOM_MM = 0.0
ODD_LEFT_MM   = 0.0
ODD_RIGHT_MM  = 0.0

# 枠消し幅（mm）：偶数ページ（2,4,6,...）
EVEN_TOP_MM    = 0.0
EVEN_BOTTOM_MM = 0.0
EVEN_LEFT_MM   = 0.0
EVEN_RIGHT_MM  = 0.0

# 横方向のスライド量（mm）：奇数ページ（+右 / -左）
ODD_SHIFT_MM   = 0.0

# 横方向のスライド量（mm）：偶数ページ（+右 / -左）
EVEN_SHIFT_MM  = 0.0

# ============================================================
# ★★ ここより下は原則編集不要（処理ロジック本体） ★★
# ============================================================

def mm_to_pt(mm: float) -> float:
    """mm を PDFのpt（1pt=1/72inch）へ変換"""
    return float(mm) * 72.0 / 25.4


def margins_for_page(idx: int):
    """ページ番号の奇偶で枠消し(mm)とスライド(mm)を返す"""
    page_no = idx + 1
    if page_no % 2 == 1:
        return (ODD_TOP_MM, ODD_BOTTOM_MM, ODD_LEFT_MM, ODD_RIGHT_MM, ODD_SHIFT_MM)
    else:
        return (EVEN_TOP_MM, EVEN_BOTTOM_MM, EVEN_LEFT_MM, EVEN_RIGHT_MM, EVEN_SHIFT_MM)


def draw_frame_erase(page: "fitz.Page", margins_pt: tuple):
    """
    出力ページに枠消し（白塗り）を適用。
    margins_pt=(top, bottom, left, right) [pt]
    """
    top_pt, bottom_pt, left_pt, right_pt = margins_pt
    rect = page.rect
    w, h = rect.width, rect.height
    white = (1, 1, 1)

    # 上
    if top_pt > 0:
        y1 = min(h, top_pt)
        page.draw_rect(fitz.Rect(0, 0, w, y1), color=white, fill=white)
    # 下
    if bottom_pt > 0:
        y0 = max(0, h - bottom_pt)
        page.draw_rect(fitz.Rect(0, y0, w, h), color=white, fill=white)
    # 左
    if left_pt > 0:
        x1 = min(w, left_pt)
        page.draw_rect(fitz.Rect(0, 0, x1, h), color=white, fill=white)
    # 右
    if right_pt > 0:
        x0 = max(0, w - right_pt)
        page.draw_rect(fitz.Rect(x0, 0, w, h), color=white, fill=white)


def process_page(src_doc: "fitz.Document", page_index: int, out_doc: "fitz.Document",
                 margins_pt: tuple, shift_pt: float):
    """
    1ページ分の処理：
     - 本文を左右方向に指定量だけスライド（show_pdf_pageで平行移動）
     - 出力ページに枠消し（白塗り）を適用
    傾き補正なし／上下方向の調整なし
    """
    page = src_doc[page_index]
    rect = page.rect
    w, h = rect.width, rect.height

    # 出力ページ（同サイズ）を作成
    out_page = out_doc.new_page(width=w, height=h)

    # スライド量（pt）を反映したターゲット矩形
    # 右へ+shift_pt / 左へ-shift_pt。ページ境界を超える部分はクリップされる。
    tgt = fitz.Rect(shift_pt, 0, w + shift_pt, h)
    out_page.show_pdf_page(tgt, src_doc, page_index)

    # 枠消し（白塗り）を上から重ねて適用
    draw_frame_erase(out_page, margins_pt)


def main():
    # 入力/出力ファイル名
    in_path = INPUT_FILENAME
    base, ext = os.path.splitext(os.path.basename(in_path))
    out_path = f"{base}_rev.pdf"

    t0 = time.time()

    # 入力PDFを開く
    src_doc = fitz.open(in_path)
    # 出力PDFを作成
    out_doc = fitz.open()

    for i in range(src_doc.page_count):
        # ページごとの設定（mm）を取得
        t_mm, b_mm, l_mm, r_mm, shift_mm = margins_for_page(i)
        # ptへ変換
        margins_pt = (mm_to_pt(t_mm), mm_to_pt(b_mm), mm_to_pt(l_mm), mm_to_pt(r_mm))
        shift_pt = mm_to_pt(shift_mm)

        # ページ処理
        process_page(src_doc, i, out_doc, margins_pt, shift_pt)

        # RAM節約のため適時GC
        if (i + 1) % 16 == 0:
            gc.collect()

    # 保存＆クローズ
    out_doc.save(out_path)
    out_doc.close()
    src_doc.close()

    elapsed = time.time() - t0
    print(f"{out_path} / {elapsed:.2f} seconds")


if __name__ == "__main__":
    main()
